In [1]:
''' Import modules '''
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from tqdm.notebook import tqdm
import numpy as np
from tqdm.notebook import tqdm
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
import heapq
from scipy.spatial import distance_matrix

## Generic utils

In [ ]:
def voronoi_partition(X, centers): 
    ''' This function returns a voronoi partition of the dataset with respect to a certain 
        set of cluster centers. 
        Inputs: 
        - X: 2d array, dataset
        - centers: set of centers
        Output: 
        - yhat: 1d array, corresponding partition'''
    distances = np.linalg.norm(X[:, np.newaxis] - centers, axis=2)
    yhat = np.argmin(distances, axis=1)
    return yhat

def compute_accuracy(y, yhat):
    ''' This function computes the accuracy of a candidate classification. 
        Inputs: 
        - y: 1d array, ground truth classification 
        - yhat: 1d array, candidate classification
        '''
    # Create label to index mappings
    y_index_map = {label: i for i, label in enumerate(np.unique(y))}
    yhat_index_map = {label: i for i, label in enumerate(np.unique(yhat))}
    if len(np.unique(y)) != len(np.unique(yhat)):
        raise ValueError("Number of unique labels in y and yhat do not match.")
    # Create a contingency table
    contingency = np.zeros((len(y_index_map), len(yhat_index_map)))
    for i in range(y.shape[0]):
        contingency[y_index_map[y[i]], yhat_index_map[yhat[i]]] += 1
    # Use the Hungarian algorithm to find the optimal assignment
    row_ind, col_ind = linear_sum_assignment(-contingency)
    accuracy = contingency[row_ind, col_ind].sum() / len(y)
    return accuracy

def kmeanspp(data, num_clusters, start_idx=None):
    """ Optimized Gonzalez's k-centers clustering on the dataset.
    Inputs:
    - data: array, data points with shape (n_samples, n_features)
    - num_clusters: int, the number of clusters to select
    - start_idx: int (optional), index of the fixed starting point. Otherwise, selected randomly
    Returns:
    - cluster_labels: 1d array, cluster assignments for each point
    - cluster_centers: 2d array, cluster centers
    - max_distance: float, achieved k centers objective value
    """
    n_samples, n_features = data.shape
    # If no starting point is provided, choose randomly
    if start_idx is None:
        start_idx = np.random.choice(n_samples)
    max_distance = 0 
    # Initialize first center
    cluster_centers = np.zeros((num_clusters, n_features))
    cluster_centers[0] = data[start_idx]
    dist_to_centers = np.linalg.norm(data - cluster_centers[0], axis=1)
    # Initialize cluster labeling
    cluster_labels = np.zeros(n_samples, dtype=int) 
    # Find remaining clusters
    for i in range(1, num_clusters):
        normalized_probs = (dist_to_centers**2) / np.sum(dist_to_centers**2)
        new_center_index = np.random.choice(len(dist_to_centers), p=normalized_probs)
        cluster_centers[i] = data[new_center_index]
        # Compute distances to the new center
        new_distances = np.linalg.norm(data - cluster_centers[i], axis=1)
        closer_to_new_center = new_distances < dist_to_centers
        # Update labels and distance to centers for points closer to the new center
        cluster_labels[closer_to_new_center] = i
        np.minimum(dist_to_centers, new_distances, out=dist_to_centers)
        # Update objective value
        max_distance = max(max_distance, np.max(dist_to_centers))
    return cluster_labels, cluster_centers, max_distance

def mcmc_kmeanspp(data, num_clusters, m, start_idx=None):
    """ 
    Optimized MCMC clustering on the dataset.
    Inputs:
    - data: array, data points with shape (n_samples, n_features)
    - num_clusters: int, the number of clusters to select
    - beta: float, temperature parameter for the softmax distribution
    - m: number of markov chain steps to use
    - start_idx: int (optional), index of the fixed starting point. Otherwise, selected randomly
    Returns:
    - cluster_labels: 1d array, cluster assignments for each point
    - cluster_centers: 2d array, cluster centers
    - max_distance: float, achieved k centers objective value
    """
    n_samples, n_features = data.shape
    # If no starting point is provided, choose randomly
    if start_idx is None:
        start_idx = np.random.choice(n_samples)
    # Initialize first center
    cluster_centers = np.zeros((num_clusters, n_features))
    cluster_centers[0] = data[start_idx]
    dist_to_centers = np.linalg.norm(data - cluster_centers[0], axis=1)
    max_distance = np.max(dist_to_centers)
    # Initialize cluster labeling
    cluster_labels = np.zeros(n_samples, dtype=int) 
    # Find remaining clusters
    for i in range(1, num_clusters):
        # Randomly draw x from the dataset.
        x_idx = np.random.choice(n_samples)
        dx = dist_to_centers[x_idx]
        # Loop from 1 to m
        for j in range(m): 
            y_idx = np.random.choice(n_samples)
            dy = dist_to_centers[y_idx]
            temp = dx 
            val = np.random.uniform(0, 1)
            if (dy*dy)/(dx*dx) > val: 
                x_idx = y_idx
                dx = dy
        cluster_centers[i] = data[x_idx]
        # Compute distances to the new center
        new_distances = np.linalg.norm(data - cluster_centers[i], axis=1)
        closer_to_new_center = new_distances < dist_to_centers
        # Update labels and distance to centers for points closer to the new center
        cluster_labels[closer_to_new_center] = i  
        np.minimum(dist_to_centers, new_distances, out=dist_to_centers)
        # Update objective value
        max_distance = max(max_distance, np.max(dist_to_centers))
    return cluster_labels, cluster_centers, max_distance

def single_linkage_clustering(X, y, num_clusters):
    # Compute initial distances and store in a heap
    dist_matrix = distance_matrix(X, X)
    heap = [(dist_matrix[i][j], (i, j)) for i in range(len(X)) for j in range(i+1, len(X))]
    heapq.heapify(heap)  # This turns the list into a heap in-place
    
    # Initially each point is its own cluster
    clusters = {i: [i] for i in range(len(X))}
    while len(clusters) > num_clusters:
        # Pop the smallest distance and get the clusters to merge
        _, (c1, c2) = heapq.heappop(heap)
        # Check if these clusters are still valid (they haven't been merged before)
        if c1 not in clusters or c2 not in clusters:
            continue
        # Merge clusters
        clusters[c1].extend(clusters[c2])
        del clusters[c2]
        # Calculate new distances between the merged cluster and all other clusters
        for c3 in clusters:
            if c3 != c1:
                new_dist = min(dist_matrix[i][j] for i in clusters[c1] for j in clusters[c3])
                heapq.heappush(heap, (new_dist, (c1, c3)))
    final_labels = np.zeros(len(X), dtype=int)
    for label, indices in enumerate(clusters.values()):
        final_labels[indices] = int(label)
    
    return final_labels

def subsample_data(X, y, m):
    """
    Takes as input feature matrix X and labels y, and returns a random subsample of m items from it.

    Parameters:
    - X: 2D array-like, feature matrix.
    - y: 1D array-like, labels.
    - m: int, the number of items to sample.

    Returns:
    - X_subsample: 2D array-like, containing m randomly sampled rows from X.
    - y_subsample: 1D array-like, corresponding labels for X_subsample.
    """
    if m > len(y):
        raise ValueError("m should not exceed the number of items in y.")
    indices = np.random.choice(len(y), m, replace=False)
    X_subsample = X[indices]
    y_subsample = y[indices]
    return X_subsample, y_subsample

def vary_m(num_clusters, num_points, global_samples, ms, X, y):
    print("generated instance", flush=True)
    labels = single_linkage_clustering(X, y, num_clusters)
    sl_accuracy = compute_accuracy(y, labels)
    print("computed base instance", flush=True)

    print(X.shape)
    print(y.shape)
    
    sl_subsample_accuracy = []
    sl_subsample_std = []
    # Run subsampled version for global_samples times
    for m in tqdm(ms, desc="m"): 
        sl_accuracy_m = []
        for rep in tqdm(range(global_samples), desc="reps", leave=False):
            X_subsample, y_subsample = subsample_data(X, y, m)
            while (len(np.unique(y_subsample)) != num_clusters): 
                X_subsample, y_subsample = subsample_data(X, y, m)
            labels = single_linkage_clustering(X_subsample, y_subsample, num_clusters)
            while (len(np.unique(labels)) != num_clusters): 
                labels = single_linkage_clustering(X_subsample, y_subsample, num_clusters)
            acc = compute_accuracy(y_subsample, labels)
            sl_accuracy_m.append(acc)
        sl_subsample_accuracy.append(np.mean(sl_accuracy_m))
        sl_subsample_std.append(np.std(sl_accuracy_m))

    return (sl_accuracy, sl_subsample_accuracy, sl_subsample_std)

''' k Centers Algorithms '''
def gonzalez(data, num_clusters, start_idx=None):
    """ Optimized Gonzalez's k-centers clustering on the dataset.
    Inputs:
    - data: array, data points with shape (n_samples, n_features)
    - num_clusters: int, the number of clusters to select
    - start_idx: int (optional), index of the fixed starting point. Otherwise, selected randomly
    Returns:
    - cluster_labels: 1d array, cluster assignments for each point
    - cluster_centers: 2d array, cluster centers
    - max_distance: float, achieved k centers objective value
    """
    n_samples, n_features = data.shape
    # If no starting point is provided, choose randomly
    if start_idx is None:
        start_idx = np.random.choice(n_samples)
    # Initialize first center
    cluster_centers = np.zeros((num_clusters, n_features))
    cluster_centers[0] = data[start_idx]
    dist_to_centers = np.linalg.norm(data - cluster_centers[0], axis=1)
    max_distance = np.max(dist_to_centers)
    # Initialize cluster labeling
    cluster_labels = np.zeros(n_samples, dtype=int) 
    # Find remaining clusters
    for i in range(1, num_clusters):
        new_center_index = np.argmax(dist_to_centers)
        cluster_centers[i] = data[new_center_index]
        # Compute distances to the new center
        new_distances = np.linalg.norm(data - cluster_centers[i], axis=1)
        closer_to_new_center = new_distances < dist_to_centers
        # Update labels and distance to centers for points closer to the new center
        cluster_labels[closer_to_new_center] = i  
        np.minimum(dist_to_centers, new_distances, out=dist_to_centers)
        # Update objective value
        max_distance = max(max_distance, np.max(dist_to_centers))
    return cluster_labels, cluster_centers, max_distance

def softmax(data, num_clusters, beta, start_idx=None):
    """ 
    Optimized Softmax clustering on the dataset.
    Inputs:
    - data: array, data points with shape (n_samples, n_features)
    - num_clusters: int, the number of clusters to select
    - beta: float, temperature parameter for the softmax distribution
    - start_idx: int (optional), index of the fixed starting point. Otherwise, selected randomly
    Returns:
    - cluster_labels: 1d array, cluster assignments for each point
    - cluster_centers: 2d array, cluster centers
    - max_distance: float, achieved k centers objective value
    """
    n_samples, n_features = data.shape
    # If no starting point is provided, choose randomly
    if start_idx is None:
        start_idx = np.random.choice(n_samples)
    # Initialize first center
    cluster_centers = np.zeros((num_clusters, n_features))
    cluster_centers[0] = data[start_idx]
    dist_to_centers = np.linalg.norm(data - cluster_centers[0], axis=1)
    max_distance = np.max(dist_to_centers)
    # Initialize cluster labeling
    cluster_labels = np.zeros(n_samples, dtype=int) 
    # Find remaining clusters
    for i in range(1, num_clusters):
        # Generate softmax distribution using normalization trick for numerical stability
        stabilized_dists = beta * (dist_to_centers - max_distance)
        probabilities = np.exp(stabilized_dists)
        probabilities /= np.sum(probabilities)
        new_center_index = np.random.choice(n_samples, p=probabilities)
        cluster_centers[i] = data[new_center_index]
        # Compute distances to the new center
        new_distances = np.linalg.norm(data - cluster_centers[i], axis=1)
        closer_to_new_center = new_distances < dist_to_centers
        # Update labels and distance to centers for points closer to the new center
        cluster_labels[closer_to_new_center] = i  
        np.minimum(dist_to_centers, new_distances, out=dist_to_centers)
        # Update objective value
        max_distance = max(max_distance, np.max(dist_to_centers))
    return cluster_labels, cluster_centers, max_distance

def mcmc_softmax(data, num_clusters, beta, m, start_idx=None):
    """ 
    Optimized MCMC clustering on the dataset.
    Inputs:
    - data: array, data points with shape (n_samples, n_features)
    - num_clusters: int, the number of clusters to select
    - beta: float, temperature parameter for the softmax distribution
    - m: number of markov chain steps to use
    - start_idx: int (optional), index of the fixed starting point. Otherwise, selected randomly
    Returns:
    - cluster_labels: 1d array, cluster assignments for each point
    - cluster_centers: 2d array, cluster centers
    - max_distance: float, achieved k centers objective value
    """
    n_samples, n_features = data.shape
    # If no starting point is provided, choose randomly
    if start_idx is None:
        start_idx = np.random.choice(n_samples)
    # Initialize first center
    cluster_centers = np.zeros((num_clusters, n_features))
    cluster_centers[0] = data[start_idx]
    dist_to_centers = np.linalg.norm(data - cluster_centers[0], axis=1)
    max_distance = np.max(dist_to_centers)
    # Initialize cluster labeling
    cluster_labels = np.zeros(n_samples, dtype=int) 
    # Find remaining clusters
    for i in range(1, num_clusters):
        # Randomly draw x from the dataset.
        x_idx = np.random.choice(n_samples)
        dx = dist_to_centers[x_idx]
        # Loop from 1 to m
        for j in range(m): 
            y_idx = np.random.choice(n_samples)
            dy = dist_to_centers[y_idx]
            temp = dx 
            val = np.random.uniform(0, 1)
            if ((beta * (dy - dx)) > np.log(val)): 
                x_idx = y_idx
                dx = dy
        cluster_centers[i] = data[x_idx]
        # Compute distances to the new center
        new_distances = np.linalg.norm(data - cluster_centers[i], axis=1)
        closer_to_new_center = new_distances < dist_to_centers
        # Update labels and distance to centers for points closer to the new center
        cluster_labels[closer_to_new_center] = i  
        np.minimum(dist_to_centers, new_distances, out=dist_to_centers)
        # Update objective value
        max_distance = max(max_distance, np.max(dist_to_centers))
    return cluster_labels, cluster_centers, max_distance

def means_vary_m(num_clusters, num_points, global_samples, sample_sizes, ms, X, y):
    kmeanspp_accuracies = []
    # Run kmeans++ for global_samples times
    for rep in tqdm(range(global_samples), desc="Global Sampling"):
        kmeanspp_labels, _, kmeanspp_obj = kmeanspp(X, num_clusters)
        while len(np.unique(kmeanspp_labels)) != len(np.unique(y)): 
            kmeanspp_labels, _, _ = kmeanspp(X, num_clusters)
        kmeanspp_accuracies.append(compute_accuracy(y, kmeanspp_labels))
    # Compute mean, median, and accuracy for kmeans++
    kmeanspp_mean_accuracy = np.mean(kmeanspp_accuracies)
    kmeanspp_std = np.std(kmeanspp_accuracies)
    # Run mcmc for each value in ms
    mcmc_mean = []
    mcmc_median = []
    mcmc_mean_accuracy = []
    mcmc_std = []
    for i, m in enumerate(tqdm(ms, desc="MCMC for different m's")):
        mcmc_obj_values = []
        mcmc_accuracies = []
        for rep in tqdm(range(sample_sizes[i]), desc=f"MCMC Sampling for m={m}", leave=False):
            mcmc_labels, _, _ = mcmc_kmeanspp(X, num_clusters, m)
            while len(np.unique(mcmc_labels)) != len(np.unique(y)): 
                mcmc_labels, _, _ = mcmc_kmeanspp(X, num_clusters, m)
            mcmc_accuracies.append(compute_accuracy(y, mcmc_labels))
        mcmc_mean_accuracy.append(np.mean(mcmc_accuracies))
        mcmc_std.append(np.std(mcmc_accuracies))
    
    return (kmeanspp_mean_accuracy, kmeanspp_std, 
            mcmc_mean_accuracy, mcmc_std)

def centers_vary_m(num_clusters, num_points, global_samples, sample_sizes, beta, ms, X, y):
    gonzalez_obj_values = []
    softmax_obj_values = []
    gonzalez_accuracies = []
    softmax_accuracies = []
    # Run softmax and gonzalez for global_samples times
    for rep in tqdm(range(global_samples), desc="Global Sampling"):
        softmax_labels, _, softmax_obj = softmax(X, num_clusters, beta)
        while len(np.unique(softmax_labels)) != len(np.unique(y)): 
            softmax_labels, _, softmax_obj = softmax(X, num_clusters, beta)
        gonzalez_labels, _, gonzalez_obj = gonzalez(X, num_clusters)
        gonzalez_obj_values.append(gonzalez_obj)
        softmax_obj_values.append(softmax_obj)
        gonzalez_accuracies.append(compute_accuracy(y, gonzalez_labels))
        softmax_accuracies.append(compute_accuracy(y, softmax_labels))
    # Compute mean, median, and accuracy for gonzalez and softmax
    gonzalez_mean = np.mean(gonzalez_obj_values)
    gonzalez_median = np.median(gonzalez_obj_values)
    gonzalez_mean_accuracy = np.mean(gonzalez_accuracies)
    softmax_mean = np.mean(softmax_obj_values)
    softmax_median = np.median(softmax_obj_values)
    softmax_mean_accuracy = np.mean(softmax_accuracies)
    gonzalez_std_accuracy = np.std(gonzalez_accuracies)
    softmax_std_accuracy = np.std(softmax_accuracies)
    # Run mcmc for each value in ms
    mcmc_mean = []
    mcmc_median = []
    mcmc_mean_accuracy = []
    mcmc_std_accuracy = []
    for i, m in enumerate(tqdm(ms, desc="MCMC for different m's")):
        mcmc_obj_values = []
        mcmc_accuracies = []
        for rep in tqdm(range(sample_sizes[i]), desc=f"MCMC Sampling for m={m}", leave=False):
            mcmc_labels, mcmc_cluster_centers, mcmc_obj = mcmc_softmax(X, num_clusters, beta, m)
            while len(np.unique(mcmc_labels)) != len(np.unique(y)): 
                mcmc_labels, _, mcmc_obj = mcmc_softmax(X, num_clusters, beta, m)
            mcmc_obj_values.append(mcmc_obj)
            mcmc_accuracies.append(compute_accuracy(y, mcmc_labels))
        mcmc_mean.append(np.mean(mcmc_obj_values))
        mcmc_median.append(np.median(mcmc_obj_values))
        mcmc_mean_accuracy.append(np.mean(mcmc_accuracies))
        mcmc_std_accuracy.append(np.std(mcmc_accuracies))
    return (gonzalez_mean_accuracy, gonzalez_std_accuracy,
            softmax_mean_accuracy, softmax_std_accuracy,
            mcmc_mean_accuracy, mcmc_std_accuracy)

## Omniglot

In [ ]:
''' Omniglot '''

def sample_by_label(labels, k, n, p, N):
    """
    Given a vector of labels, samples k unique labels and n unique indices
    with each label. Also samples a subset of rows with probability p.
    """
    unique_labels = np.unique(labels)
    chosen_labels = np.random.choice(unique_labels, k, False)
    rows = np.concatenate([np.random.choice(np.where(labels == l)[0], n, False)
                           for l in chosen_labels])

    # Generate rows_subsample
    rows_subsamples = []
    for i in range(N): 
        rows_subsamples.append(rows[np.random.rand(rows.size) < p])
    return rows, rows_subsamples

def generate(num_clusters, points, p=1, N=1):
    # Randomly choose an alphabet
    alphabet_files = sorted(glob.glob('images_background/*.csv'))
    alphabet_file = np.random.choice(alphabet_files)
    # Load that alphabet and choose a random subset of characters
    df = pd.read_csv(alphabet_file, header=None)
    # Sample rows of the .csv file by label
    labels = df.iloc[:, 0]

    # Sample examples
    [indices, indices_subsample] = sample_by_label(labels, num_clusters, points, p, N)
    output = df.iloc[indices, :]
    sub_samples = []
    for i in range(N): 
        sub_samples.append(df.iloc[indices_subsample[i], :])
    return output

def generate_feature_labels(num_clusters, points):
    ''' This function generates a random clustering instance. 
        Inputs: 
        - num_clusters: int, number of clusters/labels to sample
        - ponts: int, number of points to sample inside each cluster/label
        Outputs: 
        - X: 2d array, containing features
        - y: 1d array, containing labels
        The function will choose num_clusters random clusters/labels from the MNIST
        dataset, sample points points from each cluster, and return a dataframe
        containing the relevant data. 
    '''
    # Generate the instance
    output = generate(num_clusters, points)
    # Split into its results
    X = output.iloc[:, 1:].to_numpy()
    y = output.iloc[:, 0].to_numpy()
    return (X, y)

In [ ]:
num_clusters = 2
num_points = 20
ms = [int(i * num_points * num_clusters) for i in [.2, .4, .6, .8]]
global_samples = 1000
sample_sizes = [global_samples]*len(ms)
beta = 1
X, y = generate_feature_labels(num_clusters, num_points)

In [ ]:
(kmeanspp_mean_accuracy, kmeanspp_std, 
            means_mcmc_mean_accuracy, means_mcmc_std) = means_vary_m(num_clusters, num_points, global_samples, sample_sizes, ms, X, y)
(gonzalez_mean_accuracy, gonzalez_std_accuracy,
            softmax_mean_accuracy, softmax_std_accuracy,
            center_mcmc_mean_accuracy, center_mcmc_std_accuracy) = centers_vary_m(num_clusters, num_points, global_samples, sample_sizes, beta, ms, X, y)
(sl_accuracy, sl_subsample_accuracy, sl_subsample_std) = vary_m(num_clusters, num_points, 10, ms, X, y)

## MNIST

In [ ]:
''' MNIST '''
alphabet_file = 'mnist.csv'
def generate(num_clusters, points):
    ''' This function generates a random clustering instance. 
        Inputs: 
        - num_clusters: int, number of clusters/labels to sample
        - ponts: int, number of points to sample inside each cluster/label
        Outputs: 
        - dataframe: dataframe, containing instance data
        The function will choose num_clusters random clusters/labels from the MNIST
        dataset, sample points points from each cluster, and return a dataframe
        containing the relevant data. 
    '''
    # Choose the subset of labels to use
    labels = np.random.choice(10, num_clusters, replace=False)
    # Load the .csv data into a pandas dataframe
    df = pd.read_csv(alphabet_file, dtype=np.float64, header=None)
    # Sample 'points' many points from each of the chosen labels
    examples = [df[df.iloc[:, 0] == label].sample(n=points)
                for label in labels]
    # Concatenate the samples and return the dataframe
    output = pd.concat(examples)
    return output

def generate_feature_labels(num_clusters, points):
    ''' This function generates a random clustering instance. 
        Inputs: 
        - num_clusters: int, number of clusters/labels to sample
        - ponts: int, number of points to sample inside each cluster/label
        Outputs: 
        - X: 2d array, containing features
        - y: 1d array, containing labels
        The function will choose num_clusters random clusters/labels from the MNIST
        dataset, sample points points from each cluster, and return a dataframe
        containing the relevant data. 
    '''
    # Generate the instance
    output = generate(num_clusters, points)
    # Split into its results
    X = output.iloc[:, 1:].to_numpy()
    y = output.iloc[:, 0].to_numpy()
    return (X, y)

In [ ]:
num_clusters = 2
num_points = 250
ms = [int(i * num_points * num_clusters) for i in [.2, .4, .6, .8]]
global_samples = 1000
sample_sizes = [global_samples]*len(ms)
beta = 1
X, y = generate_feature_labels(num_clusters, num_points)

In [ ]:
(kmeanspp_mean_accuracy, kmeanspp_std, 
            means_mcmc_mean_accuracy, means_mcmc_std) = means_vary_m(num_clusters, num_points, global_samples, sample_sizes, ms, X, y)
(gonzalez_mean_accuracy, gonzalez_std_accuracy,
            softmax_mean_accuracy, softmax_std_accuracy,
            center_mcmc_mean_accuracy, center_mcmc_std_accuracy) = centers_vary_m(num_clusters, num_points, global_samples, sample_sizes, beta, ms, X, y)
(sl_accuracy, sl_subsample_accuracy, sl_subsample_std) = vary_m(num_clusters, num_points, 10, ms, X, y)

## Noisy Circles

In [ ]:
from sklearn.datasets import make_circles
def generate_nc(num_clusters, n_samples, noise=0.1, factor=0.2):
    """
    Generate a noisy circles dataset and return as a DataFrame.

    Parameters:
    - n_samples: Total number of points.
    - noise: Standard deviation of Gaussian noise added to the data.
    - factor: Scale factor between the two circles.
    - num_clusters: Number of clusters (circles). Currently supports 2.
    
    Returns:
    - df: DataFrame containing labels and coordinates.
    """
    if num_clusters != 2:
        raise ValueError("Currently only supports 2 clusters (circles).")
    
    X, y = make_circles(n_samples=n_samples, noise=noise, factor=factor)
    df = pd.DataFrame(X, columns=['X_coord', 'Y_coord'])
    df.insert(0, 'Label', y)
    X = df.loc[:, ['X_coord', 'Y_coord']].values
    y = df['Label'].to_numpy()
    return X,y

def generate_feature_labels(num_clusters, points):
    ''' This function generates a random clustering instance. 
        Inputs: 
        - num_clusters: int, number of clusters/labels to sample
        - ponts: int, number of points to sample inside each cluster/label
        Outputs: 
        - X: 2d array, containing features
        - y: 1d array, containing labels
        The function will choose num_clusters random clusters/labels from the MNIST
        dataset, sample points points from each cluster, and return a dataframe
        containing the relevant data. 
    '''
    # Generate the instance
    output = generate(num_clusters, points)
    # Split into its results
    X = output.iloc[:, 1:].to_numpy()
    y = output.iloc[:, 0].to_numpy()
    return (X, y)

In [ ]:
num_clusters = 2
num_points = 250
ms = [int(i * num_points * num_clusters) for i in [.2, .4, .6, .8]]
global_samples = 1000
sample_sizes = [global_samples]*len(ms)
beta = 1
X, y = generate_nc(num_clusters, num_points*num_clusters)

In [ ]:
(kmeanspp_mean_accuracy, kmeanspp_std, 
            means_mcmc_mean_accuracy, means_mcmc_std) = means_vary_m(num_clusters, num_points, global_samples, sample_sizes, ms, X, y)
(gonzalez_mean_accuracy, gonzalez_std_accuracy,
            softmax_mean_accuracy, softmax_std_accuracy,
            center_mcmc_mean_accuracy, center_mcmc_std_accuracy) = centers_vary_m(num_clusters, num_points, global_samples, sample_sizes, beta, ms, X, y)
(sl_accuracy, sl_subsample_accuracy, sl_subsample_std) = vary_m(num_clusters, num_points, 10, ms, X, y)

## Gaussian Mixtures

In [ ]:
def generate_gm(num_clusters, points, means=None, variances=None):
    """
    Generate Gaussian mixture dataset.

    Args:
        num_clusters (int): Number of Gaussians.
        points (int): Samples per Gaussian.
        means (list): List of mean vectors.
        variances (list): List of variances.

    Returns:
        X, y
    """
    # default to isotropic variance of 0.5 for each cluster
    if means is None:
        means = [[0,0], [1,1], [1,0], [0,1]][:num_clusters]
    if variances is None:
        variances = [0.5] * num_clusters  # isotropic variance

    X_list, y_list = [], []
    for i, mu in enumerate(means):
        # construct covariance matrix with variance[i] on the diagonal
        cov = np.eye(len(mu)) * variances[i]
        Xi = np.random.multivariate_normal(mu, cov, points)
        X_list.append(Xi)
        y_list.extend([i] * points)
    X = np.vstack(X_list)
    y = np.array(y_list)
    return X, y


In [ ]:
num_clusters = 2
num_points = 250
ms = [int(i * num_points * num_clusters) for i in [.2, .4, .6, .8]]
global_samples = 1000
sample_sizes = [global_samples]*len(ms)
beta = 1
X,y = generate_gm(num_clusters, num_points)

In [ ]:
(kmeanspp_mean_accuracy, kmeanspp_std, 
            means_mcmc_mean_accuracy, means_mcmc_std) = means_vary_m(num_clusters, num_points, global_samples, sample_sizes, ms, X, y)
(gonzalez_mean_accuracy, gonzalez_std_accuracy,
            softmax_mean_accuracy, softmax_std_accuracy,
            center_mcmc_mean_accuracy, center_mcmc_std_accuracy) = centers_vary_m(num_clusters, num_points, global_samples, sample_sizes, beta, ms, X, y)
(sl_accuracy, sl_subsample_accuracy, sl_subsample_std) = vary_m(num_clusters, num_points, 10, ms, X, y)